In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Get the dataset

In [2]:
!cp /content/gdrive/MyDrive/AI_PROJECTS/STRAWBERRY_MASKRCNN/DATASET/StrawDI_Db1.zip .

In [3]:
!unzip /content/StrawDI_Db1.zip

Streaming output truncated to the last 5000 lines.
  inflating: StrawDI_Db1/train/img/180.png  
  inflating: StrawDI_Db1/train/img/1800.png  
  inflating: StrawDI_Db1/train/img/1801.png  
  inflating: StrawDI_Db1/train/img/1802.png  
  inflating: StrawDI_Db1/train/img/1804.png  
  inflating: StrawDI_Db1/train/img/1805.png  
  inflating: StrawDI_Db1/train/img/1806.png  
  inflating: StrawDI_Db1/train/img/1807.png  
  inflating: StrawDI_Db1/train/img/1808.png  
  inflating: StrawDI_Db1/train/img/1809.png  
  inflating: StrawDI_Db1/train/img/181.png  
  inflating: StrawDI_Db1/train/img/1810.png  
  inflating: StrawDI_Db1/train/img/1811.png  
  inflating: StrawDI_Db1/train/img/1812.png  
  inflating: StrawDI_Db1/train/img/1814.png  
  inflating: StrawDI_Db1/train/img/1815.png  
  inflating: StrawDI_Db1/train/img/1816.png  
  inflating: StrawDI_Db1/train/img/1817.png  
  inflating: StrawDI_Db1/train/img/1818.png  
  inflating: StrawDI_Db1/train/img/1819.png  
  inflating: StrawDI_Db1/train/

In [4]:
! mv /content/StrawDI_Db1 /content/dataset

## Transform to Coco Format annotations

* Train dataset

In [20]:
from PIL import Image
import numpy as np
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import json
import os


def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x, y))[:3]

            # If the pixel is not black...
            if pixel != (0, 0, 0):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks


def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(np.array(sub_mask), 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords)
        segmentation = np.maximum(segmentation, 0).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    if multi_poly.bounds == ():
        return "skip"
    x, y, max_x, max_y = multi_poly.bounds
    # x = max(0, x)
    # y = max(0, y)
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation


def get_name(root, mode_folder=True):
    for root, dirs, file in os.walk(root):
        if mode_folder:
            return sorted(dirs)
        else:
            return sorted(file)


def get_annotation(mask_image_root):
    dataset = {"info": {"year": 2022, "version": "2022", "description": "Strawberry", "contributor": "Crespo", "url": "",
                        "date_created": "2022.01.29"},
               "license": {"id": 1, "url": "", "name": "Crespo"},
               "images": [],
               "annotations": [],
               "categories": []}
    class_index = {0: "strawberry"}
    for s, k in enumerate(list(class_index.keys())):
        dataset["categories"].append({"id": k, "name": class_index[k], "supercategory": "fruit"})

    is_crowd = 0

    # These ids will be automatically increased as we go
    annotation_id = 0
    image_id = 0

    # Create the annotations
    for i, root in enumerate(mask_image_root):
        try:
          #print(i)
          mask_image = Image.open(rrr + root)
          try:
            image_id = int(root.split(".")[0])
          except:
            image_id += 0
          mask_image  = mask_image.convert("RGB")
          #print(root)
          weight, height = mask_image.size
          file_name = root.split("/")[-1].split("_")[-1]
          #print(file_name)
          dataset["images"].append({"license": 1,
                                    "file_name": file_name,
                                    #"id": i,
                                    "id": image_id,
                                    "width": weight,
                                    "height": height})
          sub_masks = create_sub_masks(mask_image)
          for color, sub_mask in sub_masks.items():
              category_id = 0
              annotation = create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd)
              if annotation == "skip":
                  continue
              dataset["annotations"].append(annotation)
              annotation_id += 1
          #image_id += 1
        except:
          print(root)
          continue
    with open("train.json", "w") as f:
        json.dump(dataset, f)


rrr = "/content/dataset/train/label/"
all_root = get_name(rrr, mode_folder=False)
get_annotation(all_root)

933.png


* Val dataset

In [19]:
from PIL import Image
import numpy as np
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import json
import os


def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x, y))[:3]

            # If the pixel is not black...
            if pixel != (0, 0, 0):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks


def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(np.array(sub_mask), 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords)
        segmentation = np.maximum(segmentation, 0).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    if multi_poly.bounds == ():
        return "skip"
    x, y, max_x, max_y = multi_poly.bounds
    # x = max(0, x)
    # y = max(0, y)
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation


def get_name(root, mode_folder=True):
    for root, dirs, file in os.walk(root):
        if mode_folder:
            return sorted(dirs)
        else:
            return sorted(file)


def get_annotation(mask_image_root):
    dataset = {"info": {"year": 2022, "version": "2022", "description": "Strawberry", "contributor": "Crespo", "url": "",
                        "date_created": "2022.01.29"},
               "license": {"id": 1, "url": "", "name": "Crespo"},
               "images": [],
               "annotations": [],
               "categories": []}
    class_index = {0: "strawberry"}
    for s, k in enumerate(list(class_index.keys())):
        dataset["categories"].append({"id": k, "name": class_index[k], "supercategory": "fruit"})

    is_crowd = 0

    # These ids will be automatically increased as we go
    annotation_id = 0
    image_id = 0

    # Create the annotations
    for i, root in enumerate(mask_image_root):
        try:
          #print(i)
          mask_image = Image.open(rrr + root)
          try:
            image_id = int(root.split(".")[0])
          except:
            image_id += 0
          mask_image  = mask_image.convert("RGB")
          #print(root)
          weight, height = mask_image.size
          file_name = root.split("/")[-1].split("_")[-1]
          #print(file_name)
          dataset["images"].append({"license": 1,
                                    "file_name": file_name,
                                    #"id": i,
                                    "id": image_id,
                                    "width": weight,
                                    "height": height})
          sub_masks = create_sub_masks(mask_image)
          for color, sub_mask in sub_masks.items():
              category_id = 0
              annotation = create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd)
              if annotation == "skip":
                  continue
              dataset["annotations"].append(annotation)
              annotation_id += 1
          #image_id += 1
        except:
          print(root)
          continue
    with open("val.json", "w") as f:
        json.dump(dataset, f)


rrr = "/content/dataset/val/label/"
all_root = get_name(rrr, mode_folder=False)
get_annotation(all_root)

* Test dataset

In [5]:
from PIL import Image
import numpy as np
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import json
import os


def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x, y))[:3]

            # If the pixel is not black...
            if pixel != (0, 0, 0):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks


def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(np.array(sub_mask), 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords)
        segmentation = np.maximum(segmentation, 0).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    if multi_poly.bounds == ():
        return "skip"
    x, y, max_x, max_y = multi_poly.bounds
    # x = max(0, x)
    # y = max(0, y)
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation


def get_name(root, mode_folder=True):
    for root, dirs, file in os.walk(root):
        if mode_folder:
            return sorted(dirs)
        else:
            return sorted(file)


def get_annotation(mask_image_root):
    dataset = {"info": {"year": 2022, "version": "2022", "description": "Strawberry", "contributor": "Crespo", "url": "",
                        "date_created": "2022.01.29"},
               "license": {"id": 1, "url": "", "name": "Crespo"},
               "images": [],
               "annotations": [],
               "categories": []}
    class_index = {0: "strawberry"}
    for s, k in enumerate(list(class_index.keys())):
        dataset["categories"].append({"id": k, "name": class_index[k], "supercategory": "fruit"})

    is_crowd = 0

    # These ids will be automatically increased as we go
    annotation_id = 0
    image_id = 0

    # Create the annotations
    for i, root in enumerate(mask_image_root):
        try:
          #print(i)
          mask_image = Image.open(rrr + root)
          try:
            image_id = int(root.split(".")[0])
          except:
            image_id += 0
          mask_image  = mask_image.convert("RGB")
          #print(root)
          weight, height = mask_image.size
          file_name = root.split("/")[-1].split("_")[-1]
          #print(file_name)
          dataset["images"].append({"license": 1,
                                    "file_name": file_name,
                                    #"id": i,
                                    "id": image_id,
                                    "width": weight,
                                    "height": height})
          sub_masks = create_sub_masks(mask_image)
          for color, sub_mask in sub_masks.items():
              category_id = 0
              annotation = create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd)
              if annotation == "skip":
                  continue
              dataset["annotations"].append(annotation)
              annotation_id += 1
          #image_id += 1
        except:
          print(root)
          continue
    with open("test.json", "w") as f:
        json.dump(dataset, f)


rrr = "/content/dataset/test/label/"
all_root = get_name(rrr, mode_folder=False)
get_annotation(all_root)


## Copy Jsons to Drive 

In [ ]:
!cp train.json /content/gdrive/Shareddrives/Universidad/CV_strawberry_project/strawberry_detection_fruit/dataset/COCO_FORMAT_ANNOTATIONS
!cp val.json /content/gdrive/Shareddrives/Universidad/CV_strawberry_project/strawberry_detection_fruit/dataset/COCO_FORMAT_ANNOTATIONS
!cp test.json /content/gdrive/Shareddrives/Universidad/CV_strawberry_project/strawberry_detection_fruit/dataset/COCO_FORMAT_ANNOTATIONS

## Visualize the masks from the COCO formar annotations.

In [6]:
import IPython
import os
import json
import random
import numpy as np
import requests
from io import BytesIO
from math import trunc
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

# Load the dataset json
class CocoDataset():
    def __init__(self, annotation_path, image_dir):
        self.annotation_path = annotation_path
        self.image_dir = image_dir
        self.colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                       'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                       'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                       'magenta', 'sienna', 'maroon']

        json_file = open(self.annotation_path)
        self.coco = json.load(json_file)
        json_file.close()

        self.process_info()
        self.process_licenses()
        self.process_categories()
        self.process_images()
        self.process_segmentations()

    def display_info(self):
        print('Dataset Info:')
        print('=============')
        if self.info is None:
            return
        for key, item in self.info.items():
            print('  {}: {}'.format(key, item))

        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        print('')

    def display_licenses(self):
        print('Licenses:')
        print('=========')

        if self.licenses is None:
            return
        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(
                        req, str(req_type)))
            print('')
        print('')

    def display_categories(self):
        print('Categories:')
        print('=========')
        for sc_key, sc_val in self.super_categories.items():
            print('  super_category: {}'.format(sc_key))
            for cat_id in sc_val:
                print('    id {}: {}'.format(
                    cat_id, self.categories[cat_id]['name']))
            print('')

    def display_image(self, image_id, show_polys=True, show_bbox=True, show_crowds=True, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))

        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))

        # Open the image
        if use_url:
            image_path = image['coco_url']
            response = requests.get(image_path)
            image = PILImage.open(BytesIO(response.content))

        else:
            # image_path = os.path.join(self.image_dir, image['file_name'])
            image_path = "{}/{}".format(self.image_dir, image['file_name'])
            image = PILImage.open(image_path)

        # Calculate the size and adjusted display size
        max_width = 600
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height

        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        bbox_categories = {}
        print('  segmentations ({}):'.format(
            len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            polygons_list = []
            if segm['iscrowd'] != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(
                            trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height)
                                      * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append(
                                {'x': x_start, 'y': y_start, 'width': 1, 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append(
                                {'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})

                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1  # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(
                                    full_lines_to_insert * adjusted_ratio)
                                rle_list.append(
                                    {'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})

                            # Insert bottom line
                            rle_list.append(
                                {'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list
            else:
                # Add the polygon segmentation
                for segmentation_points in segm['segmentation']:
                    segmentation_points = np.multiply(
                        segmentation_points, adjusted_ratio).astype(int)
                    polygons_list.append(
                        str(segmentation_points).lstrip('[').rstrip(']'))
            polygons[segm['id']] = polygons_list
            if i < len(self.colors):
                poly_colors[segm['id']] = self.colors[i]
            else:
                poly_colors[segm['id']] = 'white'

            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] +
                           bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[segm['id']] = str(
                bbox_points).lstrip('[').rstrip(']')
            bbox_categories[segm['id']] = self.categories[segm['category_id']]
            # Print details
            print('    {}:{}:{}'.format(
                segm['id'], poly_colors[segm['id']], self.categories[segm['category_id']]))

        # Draw segmentation polygons on image
        html = '<div class="container" style="position:relative;">'
        html += '<img src="{}" style="position:relative;top:0px;left:0px;width:{}px;">'.format(
            image_path, adjusted_width)
        html += '<div class="svgclass"><svg width="{}" height="{}">'.format(
            adjusted_width, adjusted_height)

        if show_polys:
            for seg_id, points_list in polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for points in points_list:
                    html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5" />'.format(
                        points, fill_color, stroke_color)

        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(
                        x, y, w, h, fill_color, stroke_color)

        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                x, y = [int(i) for i in points.split()[:2]]
                html += '<text x="{}" y="{}" fill="yellow">{}</text>'.format(
                    x, y, bbox_categories[seg_id]["name"])
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(
                    points, fill_color, stroke_color)

        html += '</svg></div>'
        html += '</div>'
        html += '<style>'
        html += '.svgclass { position:absolute; top:0px; left:0px;}'
        html += '</style>'
        return html

    def process_info(self):
        self.info = self.coco.get('info')

    def process_licenses(self):
        self.licenses = self.coco.get('licenses')

    def process_categories(self):
        self.categories = {}
        self.super_categories = {}
        for category in self.coco['categories']:
            cat_id = category['id']
            super_category = category['supercategory']

            # Add category to the categories dict
            if cat_id not in self.categories:
                self.categories[cat_id] = category
            else:
                print("ERROR: Skipping duplicate category id: {}".format(category))

            # Add category to super_categories dict
            if super_category not in self.super_categories:
                # Create a new set with the category id
                self.super_categories[super_category] = {cat_id}
            else:
                self.super_categories[super_category] |= {
                    cat_id}  # Add category id to the set

    def process_images(self):
        self.images = {}
        for image in self.coco['images']:
            image_id = image['id']
            if image_id in self.images:
                print("ERROR: Skipping duplicate image id: {}".format(image))
            else:
                self.images[image_id] = image

    def process_segmentations(self):
        self.segmentations = {}
        for segmentation in self.coco['annotations']:
            image_id = segmentation['image_id']
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append(segmentation)

### See labels from train dataset

In [21]:
annotation_path = "/content/train.json"
image_dir = "/content/dataset/train/label"

coco_dataset = CocoDataset(annotation_path, image_dir)
coco_dataset.display_info()
coco_dataset.display_licenses()
coco_dataset.display_categories()

Dataset Info:
  year: 2022
  version: 2022
  description: Strawberry
  contributor: Crespo
  url: 
  date_created: 2022.01.29

Licenses:
Categories:
  super_category: fruit
    id 0: strawberry



In [22]:
for k, v in coco_dataset.images.items():
    print(k, v)

1 {'license': 1, 'file_name': '1.png', 'id': 1, 'width': 1008, 'height': 756}
10 {'license': 1, 'file_name': '10.png', 'id': 10, 'width': 1008, 'height': 756}
100 {'license': 1, 'file_name': '100.png', 'id': 100, 'width': 1008, 'height': 756}
1000 {'license': 1, 'file_name': '1000.png', 'id': 1000, 'width': 1008, 'height': 756}
1001 {'license': 1, 'file_name': '1001.png', 'id': 1001, 'width': 1008, 'height': 756}
1003 {'license': 1, 'file_name': '1003.png', 'id': 1003, 'width': 1008, 'height': 756}
1004 {'license': 1, 'file_name': '1004.png', 'id': 1004, 'width': 1008, 'height': 756}
1005 {'license': 1, 'file_name': '1005.png', 'id': 1005, 'width': 1008, 'height': 756}
1006 {'license': 1, 'file_name': '1006.png', 'id': 1006, 'width': 1008, 'height': 756}
1007 {'license': 1, 'file_name': '1007.png', 'id': 1007, 'width': 1008, 'height': 756}
1008 {'license': 1, 'file_name': '1008.png', 'id': 1008, 'width': 1008, 'height': 756}
1009 {'license': 1, 'file_name': '1009.png', 'id': 1009, 'wid

In [24]:
html = coco_dataset.display_image(990, use_url=False)
IPython.display.HTML(html)

Image:
  license: 1
  file_name: 990.png
  id: 990
  width: 1008
  height: 756
  segmentations (5):
    16182:blue:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    16183:purple:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    16184:red:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    16185:green:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    16186:orange:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}


### See labels from val dataset

In [25]:
annotation_path = "/content/val.json"
image_dir = "/content/dataset/val/label"

coco_dataset = CocoDataset(annotation_path, image_dir)
coco_dataset.display_info()
coco_dataset.display_licenses()
coco_dataset.display_categories()

Dataset Info:
  year: 2022
  version: 2022
  description: Strawberry
  contributor: Crespo
  url: 
  date_created: 2022.01.29

Licenses:
Categories:
  super_category: fruit
    id 0: strawberry



In [26]:
for k, v in coco_dataset.images.items():
    print(k, v)

1002 {'license': 1, 'file_name': '1002.png', 'id': 1002, 'width': 1008, 'height': 756}
1029 {'license': 1, 'file_name': '1029.png', 'id': 1029, 'width': 1008, 'height': 756}
108 {'license': 1, 'file_name': '108.png', 'id': 108, 'width': 1008, 'height': 756}
1099 {'license': 1, 'file_name': '1099.png', 'id': 1099, 'width': 1008, 'height': 756}
1104 {'license': 1, 'file_name': '1104.png', 'id': 1104, 'width': 1008, 'height': 756}
1112 {'license': 1, 'file_name': '1112.png', 'id': 1112, 'width': 1008, 'height': 756}
113 {'license': 1, 'file_name': '113.png', 'id': 113, 'width': 1008, 'height': 756}
1147 {'license': 1, 'file_name': '1147.png', 'id': 1147, 'width': 1008, 'height': 756}
117 {'license': 1, 'file_name': '117.png', 'id': 117, 'width': 1008, 'height': 756}
1215 {'license': 1, 'file_name': '1215.png', 'id': 1215, 'width': 1008, 'height': 756}
1272 {'license': 1, 'file_name': '1272.png', 'id': 1272, 'width': 1008, 'height': 756}
1274 {'license': 1, 'file_name': '1274.png', 'id': 1

In [27]:
html = coco_dataset.display_image(967, use_url=False)
IPython.display.HTML(html)

Image:
  license: 1
  file_name: 967.png
  id: 967
  width: 1008
  height: 756
  segmentations (3):
    569:blue:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    570:purple:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    571:red:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}


### See labels from test dataset

In [28]:
annotation_path = "/content/test.json"
image_dir = "/content/dataset/test/label"

coco_dataset = CocoDataset(annotation_path, image_dir)
coco_dataset.display_info()
coco_dataset.display_licenses()
coco_dataset.display_categories()

Dataset Info:
  year: 2022
  version: 2022
  description: Strawberry
  contributor: Crespo
  url: 
  date_created: 2022.01.29

Licenses:
Categories:
  super_category: fruit
    id 0: strawberry



In [29]:
for k, v in coco_dataset.images.items():
    print(k, v)

1017 {'license': 1, 'file_name': '1017.png', 'id': 1017, 'width': 1008, 'height': 756}
1026 {'license': 1, 'file_name': '1026.png', 'id': 1026, 'width': 1008, 'height': 756}
1028 {'license': 1, 'file_name': '1028.png', 'id': 1028, 'width': 1008, 'height': 756}
1033 {'license': 1, 'file_name': '1033.png', 'id': 1033, 'width': 1008, 'height': 756}
1080 {'license': 1, 'file_name': '1080.png', 'id': 1080, 'width': 1008, 'height': 756}
1087 {'license': 1, 'file_name': '1087.png', 'id': 1087, 'width': 1008, 'height': 756}
1108 {'license': 1, 'file_name': '1108.png', 'id': 1108, 'width': 1008, 'height': 756}
1135 {'license': 1, 'file_name': '1135.png', 'id': 1135, 'width': 1008, 'height': 756}
1151 {'license': 1, 'file_name': '1151.png', 'id': 1151, 'width': 1008, 'height': 756}
1155 {'license': 1, 'file_name': '1155.png', 'id': 1155, 'width': 1008, 'height': 756}
1156 {'license': 1, 'file_name': '1156.png', 'id': 1156, 'width': 1008, 'height': 756}
1218 {'license': 1, 'file_name': '1218.png'

In [31]:
html = coco_dataset.display_image(995, use_url=False)
IPython.display.HTML(html)

Image:
  license: 1
  file_name: 995.png
  id: 995
  width: 1008
  height: 756
  segmentations (8):
    1124:blue:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    1125:purple:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    1126:red:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    1127:green:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    1128:orange:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    1129:salmon:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    1130:pink:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
    1131:gold:{'id': 0, 'name': 'strawberry', 'supercategory': 'fruit'}
